# A New QR Algorithm

The step we use is given by $$A(nb+1:n,nb+1:n)\leftarrow A(nb+1:n, nb+1:n)-A(nb+1:n,1:nb)\times A(1:nb,nb+1:n)$$

In [14]:
using LinearAlgebra
I(3)[2,:]

3-element Vector{Bool}:
 0
 1
 0

In [15]:
#so in code form we find
function NewHouseholderQR(A)
    n, m = size(A)
    R = copy(A)  # Work on a copy to avoid modifying input
    
    for k in 1:min(n-1, m)
        # Extract the column we're working on (from row k to the end)
        x = R[k:end, k]
        
        # Compute norm
        norm_x = norm(x)
        if norm_x < 1e-15  # Already essentially zero, skip
            continue
        end
        
        # Create e1 for this submatrix
        e1 = zeros(eltype(x), length(x))
        e1[1] = 1
        
        # For Householder: we want H*x = α*e1 where |α| = ||x||
        # Choose sign to avoid cancellation: use -sign(x[1])*||x||*e1
        # This means v = x - sign(x[1])*||x||*e1
        if abs(x[1]) < 1e-15
            # If x[1] is essentially zero, use e1 directly
            sigma = -one(eltype(x))
        else
            # Use the phase of x[1]: sigma = x[1]/|x[1]|, then v = x - sigma*norm_x*e1
            sigma = x[1] / abs(x[1])
        end
        
        # Compute Householder vector: v = x - sigma * norm_x * e1
        # The negative sign ensures numerical stability (avoiding cancellation)
        v = x - sigma * norm_x * e1
        
        # Normalize v
        norm_v = norm(v)
        if norm_v < 1e-15
            # This means x was already aligned with e1 (or -e1), so no reflection needed
            continue
        end
        v = v / norm_v
        
        # Apply Householder reflection: H = I - 2*v*v' (using adjoint for complex)
        # Update the submatrix: R[k:end, k:end] = (I - 2*v*v') * R[k:end, k:end]
        # Create identity matrix of appropriate size
        I_sub = Matrix{eltype(v)}(I, length(v), length(v))
        H_sub = I_sub - 2 * v * v'  # Householder reflector (v' is adjoint for complex)
        
        # Apply to remaining columns
        R[k:end, k:end] = H_sub * R[k:end, k:end]
        
        # Zero out below diagonal in column k (should already be done, but ensure it)
        R[(k+1):end, k] .= zero(eltype(R))
    end
    
    return R
end

NewHouseholderQR (generic function with 1 method)

In [16]:
using LinearAlgebra

function GenerateRandUnitaries(n)
    [Matrix(qr(rand(ComplexF64, (2^i,2^i))).Q) for i=1:n]
end

n = 2
UList = GenerateRandUnitaries(n)
UList_2 = deepcopy(UList)

JuliaQ = Matrix(qr(UList[n]).R)
OurQ = NewHouseholderQR(UList_2[n])
display(JuliaQ)
display(UList[n])
display(OurQ)

4×4 Matrix{ComplexF64}:
 1.0+0.0im  -8.32667e-17+5.55112e-17im  …  -2.22045e-16+0.0im
 0.0+0.0im           1.0+0.0im             -5.55112e-17+2.77556e-17im
 0.0+0.0im           0.0+0.0im              1.11022e-16-2.22045e-16im
 0.0+0.0im           0.0+0.0im                      1.0+0.0im

4×4 Matrix{ComplexF64}:
    -0.2925-0.359441im  -0.143679-0.322041im  …   0.506781+0.293436im
 -0.0753102-0.378907im  -0.264977+0.705305im      0.155178-0.217856im
  -0.489689-0.295738im  -0.216575-0.434126im     -0.499732-0.350845im
  -0.435411-0.345202im   0.162385+0.215032im     0.0149372+0.460953im

4×4 Matrix{ComplexF64}:
 -0.631183-0.775634im  …   1.38778e-17+9.71445e-17im
       0.0+0.0im           1.66533e-16+2.77556e-17im
       0.0+0.0im          -5.55112e-17+3.60822e-16im
       0.0+0.0im             -0.607575-0.794263im